# Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0 and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible.

In [11]:
import numpy as np
import random
from abc import ABC, abstractmethod

In [12]:
class Game():
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.maze = (np.random.rand(self.height, self.width) * 10).astype('int')
        self.start = self.maze[0][0]
        self.end = self.maze[self.height - 1][self.width - 1]
        self.game_complete = False
        # Pointer
        self.pointer_x = 0
        self.pointer_y = 0
        self.pointer_val = self.maze[0][0]
        self.pointer = f"Currently at [{0}, {0}], of cost {self.start}"
        # Records
        self.steps_taken = 0
        self.time_spent = 0
    

    #Move the pointer from its current position, to the position passed in as parameters (x, y)
    def move_pointer(self, x_pos, y_pos):
        # Check if x_pos and y_pos are legal (step size of 1 unit, that does not exceed maze boundary)
        if(abs(x_pos - self.pointer_x) > 1 or (x_pos >= self.width or x_pos < 0)):
            print("X Step too large")
            return
        if(abs(y_pos - self.pointer_y) > 1 or (y_pos >= self.height or y_pos < 0)):
            print("Y Step too large")
            return
        # Check that step is not diagonal (i.e. it is not the case that both x and y positions have changed)
        if((x_pos != self.pointer_x) and (y_pos != self.pointer_y)):
            print("Cannot move diagonally!")
            return
        # If all is legal, make the step
        # Update records
        self.steps_taken += 1
        self.time_spent += self.pointer_val
        # Move pointer
        self.pointer_x = x_pos
        self.pointer_y = y_pos
        self.pointer_val = self.maze[y_pos][x_pos]
        self.pointer = f"Currently at [{x_pos}, {y_pos}], of cost {self.pointer_val}"
        print(f"{self.pointer}")
        # Check if after the move, the agent has won the game.
        if(self.pointer_x == (self.width - 1) and self.pointer_y == (self.height - 1)):
            self.game_complete = True
            print("Won the game! 🥳")
    

    #Reset the game
    def reset(self):
        print("Resetting game...")
        self.game_complete = False
        self.pointer_x = 0
        self.pointer_y = 0
        self.steps_taken = 0
        self.time_spent = 0
        self.pointer = f"Currently at [{0}, {0}], of cost {self.start}"
        print(self.pointer)
    
    def get_cost(self, x_pos, y_pos):
        return self.maze[y_pos][x_pos]


    #Given a point on the grid (x, y), return an array of neighboring coordinates [[x1, y1], [x2, y2], [x3, y3]...etc.]
    def get_neighbors(self, x_pos, y_pos):
        x_plus = x_pos + 1 if x_pos < (self.width - 1) else None
        x_minus = x_pos - 1 if x_pos > 0 else None
        y_plus = y_pos + 1 if y_pos < (self.height - 1) else None
        y_minus = y_pos - 1 if y_pos > 0 else None
        
        above = [x_pos, y_minus]
        below = [x_pos, y_plus]
        left = [x_minus, y_pos]
        right = [x_plus, y_pos]

        neighbors = [above, below, left, right]

        result = [ n for n in neighbors if (n[0] != None and n[1] != None)]

        return result




In [13]:
#Test game
game = Game(width=5, height=5)
print(f"{game.maze}\n")

print("After a move to x=1, y=0")
game.move_pointer(x_pos=1, y_pos=0)

[[7 0 1 6 9]
 [2 5 4 1 3]
 [6 4 8 4 2]
 [4 8 7 1 1]
 [8 6 5 8 2]]

After a move to x=1, y=0
Currently at [1, 0], of cost 0


# Agents

After implementing the game, we are tasked with developing agents to play it. All agents will conform to the Agent abstract base class. They must all take a game (type Game) into their constructor, and implement a method play_game() which plays the game that the agent was initialized with, as well as the score() method, which prints the agent's performance in completing the game.

In [14]:
class Agent(ABC):
    @abstractmethod
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0
        
    @abstractmethod
    def play_game(self):
        pass
    
    @abstractmethod
    def score(self):
        pass

## Random Agent

The first agent we need to develop is a "basline agent". The baseline agent needs to be superior to a random agent in reaching the end of the game. We implement a random agent to test their performance for comparison's sake.

In [15]:
class Random_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0
    
    def play_game(self):
        self.game.reset()
        # While the game is not complete...
        while(not self.game.game_complete):
            # Determine if we're moving along the y axis (if not, the step is along the x axis)
            move_y = random.choice((0, 1))
            # Determine if our direction is negative or positive (e.g. backwards along the y axis vs forwards along the y axis)
            direction = random.choice((-1, 1))

            # If the move is along the x axis, ensure the move does not exceed the boundaries of the maze (in that case, go back to start of loop)
            # Then, move in the random direction alongside the x axis
            if(move_y == 0):
                new_x = self.game.pointer_x + direction
                new_y = self.game.pointer_y
                if(new_x >= self.game.width or new_x < 0):
                    continue
                # print("Moving x")
                self.game.move_pointer(x_pos=new_x, y_pos=new_y)
            #Do the same if it were determined that the move should be along the y axis...
            else:
                new_x = self.game.pointer_x
                new_y = self.game.pointer_y + direction
                if(new_y >= self.game.height or new_y < 0):
                    continue
                # print("Moving y")
                self.game.move_pointer(x_pos=new_x, y_pos=new_y)
        
        self.steps_taken = self.game.steps_taken
        self.time_spent = self.game.time_spent
        return

    def score(self):
        print(f"\nTime spent: {self.time_spent}, Steps taken: {self.steps_taken}")


In [16]:
# game2 = Game(width=6, height=4)
# print(f"{game2.maze}\n")
random_agent = Random_Agent(game)
random_agent.play_game()
random_agent.score()

Resetting game...
Currently at [0, 0], of cost 7
Currently at [1, 0], of cost 0
Currently at [2, 0], of cost 1
Currently at [2, 1], of cost 4
Currently at [2, 2], of cost 8
Currently at [3, 2], of cost 4
Currently at [3, 3], of cost 1
Currently at [2, 3], of cost 7
Currently at [2, 2], of cost 8
Currently at [2, 1], of cost 4
Currently at [2, 0], of cost 1
Currently at [2, 1], of cost 4
Currently at [2, 2], of cost 8
Currently at [3, 2], of cost 4
Currently at [3, 1], of cost 1
Currently at [2, 1], of cost 4
Currently at [1, 1], of cost 5
Currently at [1, 0], of cost 0
Currently at [1, 1], of cost 5
Currently at [1, 2], of cost 4
Currently at [2, 2], of cost 8
Currently at [1, 2], of cost 4
Currently at [1, 3], of cost 8
Currently at [2, 3], of cost 7
Currently at [3, 3], of cost 1
Currently at [2, 3], of cost 7
Currently at [2, 4], of cost 5
Currently at [1, 4], of cost 6
Currently at [1, 3], of cost 8
Currently at [1, 2], of cost 4
Currently at [2, 2], of cost 8
Currently at [2, 1], 

# Baseline Agent

Our baseline agent will move diagonally towards the bottom right. Once it reaches an edge, it then moves (either downwards or across to the left, depending on maze dimensions), until it reaches the end point

In [17]:
class Baseline_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0
    

    def play_game(self):
        self.game.reset()
        while(not self.game.game_complete):
            # If we reached the bottom edge, move across to the right
            if (self.game.pointer_y == self.game.height - 1):
                new_x = self.game.pointer_x + 1
                new_y = self.game.pointer_y
                self.game.move_pointer(x_pos=(new_x), y_pos=(new_y))

            # Else if we reached the right edge, move downwards
            elif (self.game.pointer_x == self.game.width - 1):
                new_x = self.game.pointer_x
                new_y = self.game.pointer_y + 1
                self.game.move_pointer(x_pos=(new_x), y_pos=(new_y))

            # Else, move diagonally, by going through two steps
            else:
                new_x_1 = self.game.pointer_x + 1
                new_y_1 = self.game.pointer_y
                self.game.move_pointer(x_pos=(new_x_1), y_pos=(new_y_1))
                new_x_2 = self.game.pointer_x
                new_y_2 = self.game.pointer_y + 1
                self.game.move_pointer(x_pos=(new_x_2), y_pos=(new_y_2))

        self.steps_taken = self.game.steps_taken
        self.time_spent = self.game.time_spent
        return
    
    def score(self):
        print(f"\nTime spent: {self.time_spent}, Steps taken: {self.steps_taken}")
            

In [18]:
# game3 = Game(width=20, height=4)
# print(f"{game3.maze}\n")
baseline_agent = Baseline_Agent(game)
baseline_agent.play_game()
baseline_agent.score()

Resetting game...
Currently at [0, 0], of cost 7
Currently at [1, 0], of cost 0
Currently at [1, 1], of cost 5
Currently at [2, 1], of cost 4
Currently at [2, 2], of cost 8
Currently at [3, 2], of cost 4
Currently at [3, 3], of cost 1
Currently at [4, 3], of cost 1
Currently at [4, 4], of cost 2
Won the game! 🥳

Time spent: 25, Steps taken: 8


# Dijkstra's Algorithm

We build an agent that plays the game using Dijkstra's algorithm.

WHILE verticies remain unvisted
    

In [19]:
class Dijkstra_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
        self.steps_taken = 0
        self.time_spent = 0

        self.number_of_nodes = self.game.width * self.game.height
        self.current_node = f"{0},{0}"
        self.visted_nodes = []
        self.path = []

        #Initialize dictionary info, whose schema looks like:
        #      {“node”: {previous: “node”, cost: 0}...}
        # Where "node" is a string representation of the coordinates of a cell on the grid... e.g. "0,0" for [0, 0]
        self.info = {}
        # Fill info with values given the game's dimensions
        for w in range(self.game.width):
            for h in range(self.game.height):
                if(w == 0 and h == 0 ):
                    self.info[f"{w},{h}"] = {"prev": None, "cost": 0}
                else:
                    self.info[f"{w},{h}"] = {"prev": None, "cost": float("inf")}
    

    def play_game(self):
        self.game.reset()
        # Fills in self.info "cost" and "prev" values for each node
        self.explore()
        # Uses self.info to build up the quickest path to endpoint
        self.build_path()
        # Play the game by following the path
        for n in self.path:
            move_cor = self.string_to_coor(n)
            self.game.move_pointer(x_pos=move_cor[0], y_pos=move_cor[1])
        
        self.steps_taken = self.game.steps_taken
        self.time_spent = self.game.time_spent
        

    def build_path(self):
        # Initialize "node" to equal the end point
        node = f"{self.game.width - 1},{self.game.height - 1}"
        # Working backwards, check the value for "prev" in self.info, corresponding to the node defined above
        # This will give us the quickest path going backwards, starting from the end, and reaching the start
        # Insert every encountered node on this path going backwards, to the front of self.path. 
        # The end result is a list of corrodinates for the quickest path, from start to end
        while not node == f"{0},{0}":
            self.path.insert(0, node)
            node = self.info[node]["prev"]


    def explore(self):
        #Base case - if we've visted all nodes, we're done
        if len(self.visted_nodes) >= self.number_of_nodes:
            return

        #Convert self.current_node into numerical format using self.string_to_coor.. e.g. "0,0" -> [0, 0]
        current_node_numb = self.string_to_coor(self.current_node)
        #Get all the neighbors of self.current_node (helper function that outputs all neighbors is in Game class)
        current_neighbors = self.game.get_neighbors(current_node_numb[0], current_node_numb[1])

        #For each neighbor that has not been previously visited (i.e. is not in self.visited_nodes) do the following:
        # - Initialize a variable called "current_cost". 
        #   Its value is the current_node's cost as specified in self.info (total cost from start to current_node), plus the cost of traveling to this neighbor from current_node
        # - If value for the variable "current_cost" is less than the neighbor's cost as specified in self.info (total cost from start to that neighbor), 
        #   set info["neighbor"]["cost"] to current_cost, and info["neighbor"]["prev"] to current_node
        for n in current_neighbors:
            if not n in self.visted_nodes:
                current_cost = self.info[self.current_node]["cost"] + self.game.get_cost(x_pos=n[0], y_pos=n[1])
                if current_cost <  self.info[f"{n[0]},{n[1]}"]["cost"]:
                    self.info[f"{n[0]},{n[1]}"]["cost"] = current_cost
                    self.info[f"{n[0]},{n[1]}"]["prev"] = self.current_node
        
        # Tick current_node off by adding it to self.visited_nodes
        self.visted_nodes.append(self.current_node)

        # Set current_node to equal to the non visited node with the lowest cost as specified in info (total cost from start to that node)
        self.update_current_node()

        #Recursivley call explore again
        self.explore()


    def update_current_node(self):
        if len(self.visted_nodes) >= self.number_of_nodes:
            return

        least_cost = float("inf")
        least_cost_node = ""

        for n in self.info:
            if (not n in self.visted_nodes and self.info[n]["cost"] < least_cost):
                least_cost = self.info[n]["cost"]
                least_cost_node = n
        
        self.current_node = least_cost_node


    #Takes a string s, and returns an array of the corrdinates... e.g. "0,0" -> [0, 0]
    def string_to_coor(self, input):
        split_string = input.split(",")
        for i in range(len(split_string)):
            split_string[i] = int(split_string[i])
        return split_string

    
    def score(self):
        print(f"\nTime spent: {self.time_spent}, Steps taken: {self.steps_taken}")

            



In [20]:
# game4 = Game(width=6, height=4)
# print(f"{game4.maze}\n")
d_agent = Dijkstra_Agent(game)
d_agent.play_game()
d_agent.score()

Resetting game...
Currently at [0, 0], of cost 7
Currently at [1, 0], of cost 0
Currently at [2, 0], of cost 1
Currently at [2, 1], of cost 4
Currently at [3, 1], of cost 1
Currently at [3, 2], of cost 4
Currently at [3, 3], of cost 1
Currently at [4, 3], of cost 1
Currently at [4, 4], of cost 2
Won the game! 🥳

Time spent: 14, Steps taken: 8
